# Create prospective LCI databases

This notebook creates prosepctive LCI databases based on lithium supply scenario data, project-specific LCIs, ecoinvent database, and premise.

The following steps are involved:

1- Create the configuration file for integrating IEA electricity scenarios via premise

2- Create background LCI databases based on IAM and IEA scenarios

3- Create foreground LCI databases with lithium supply scenarios



In [1]:
import pandas as pd
import yaml
import brightway2 as bw
from premise import *
from premise.utils import (load_database, eidb_label)
from premise.new_database import _prepare_database
from premise.brightway2 import write_brightway_database
import wurst
from datapackage import Package
from pathlib import Path
from itertools import chain, product
import copy
from utils import(
    mapping_scenario_variables_to_ids,
    populate_prod_pathaways,
    relink_to_regionalized_water
    )

# Define data paths
PROJECT_PATH = Path("../")
SCENARIO_DATA_PATH = PROJECT_PATH / "scenario_data"
CONFIG_FILE_PATH = PROJECT_PATH / "configuration_file"
DATAPACKAGE_PATH = PROJECT_PATH / 'datapackage.json'

LITHIUM_SCENARIO_DATA_PATH = SCENARIO_DATA_PATH / "external" / "lithium_scenario_data_with_others_22-01-2025.csv"
ELECTRICITY_SCENARIO_DATA_PATH = SCENARIO_DATA_PATH / "scenario_data_electricity.csv"

MAP_LITHIUM_LCIS_PATH = SCENARIO_DATA_PATH / "map_litihum_projects_to_lci.xlsx"
MAP_ELECTRICITY_LCIS_PATH = SCENARIO_DATA_PATH / "map_electricity_tech_to_lci.xlsx"

In [ ]:
# Set-up BW project
BW_PROJECT = 'lithium_brine'
bw.projects.set_current(BW_PROJECT)

EI_DB = "ecoinvent-3.10-cutoff"
LI_PROJECTS_DB = "lithium_brine_projects"
WATER_BIO_DB = "biosphere water regionalized"
water_flows_regionalized = [act for act in bw.Database(WATER_BIO_DB)]

PREMISE_KEY = 'XXXXXXXXXXXXXXX' # <- insert premise key here (get it from developer)

## 1. Create background LCI databases

Here we create the background LCI databases using *premise* and integrate electricity mixes from IEA

### 1.1 Create configuration file for IEA electricity scenarios for use in premise

In [ ]:
# Import electricity scenario data and mapping file between electricity variables and LCI datasets
scenario_data_electricity = pd.read_csv(ELECTRICITY_SCENARIO_DATA_PATH)
map_elec_tech_to_lci = pd.read_excel(MAP_ELECTRICITY_LCIS_PATH)
map_elec_tech_to_lci = dict(chain(zip(map_elec_tech_to_lci['Power generation technology'], map_elec_tech_to_lci['LCI dataset'])))

# Map scenario variables to variable ID
elec_scenario_variables = mapping_scenario_variables_to_ids(scenario_data_electricity)

# Create config file for electricity
config_file_elec = {}

# Define market names
ELEC_HIGH_VOLT_MARKET = "market for electricity, high voltage, IEA"
ELEC_MEDIUM_VOLT_MARKET = "market for electricity, medium voltage, IEA"
ELEC_LOW_VOLT_MARKET = "market for electricity, low voltage, IEA"

# Populate production pathways
config_file_elec["production pathways"] = {}

for var_id in elec_scenario_variables:

    if var_id == "Low to medium":
        # Conversion of low voltage to medium voltage
        var_lci = ELEC_MEDIUM_VOLT_MARKET
        var_product = "electricity, medium voltage"
        in_ecoinvent = False
        regionalize = False
        new_dataset = True

    elif var_id == "Medium to high":
        # Conversion of medium voltage to high voltage
        var_lci = ELEC_HIGH_VOLT_MARKET
        var_product = "electricity, high voltage"
        in_ecoinvent = False
        regionalize = False
        new_dataset = True

    else:
        var_lci = map_elec_tech_to_lci[var_id]
        regionalize = False
        new_dataset = False
        in_ecoinvent = True

        if var_id == "Solar PV":
            var_product = "electricity, low voltage"
        else:
            var_product = "electricity, high voltage"
        
    populate_prod_pathaways(
        config_file=config_file_elec, 
        var_id=var_id, 
        var_lci=var_lci, 
        var_product=var_product, 
        in_ecoinvent=in_ecoinvent, 
        regionalize=regionalize,
        new_dataset=new_dataset, 
        var=elec_scenario_variables[var_id]
    )

# Populate markets
config_file_elec["markets"] = [

    # High voltage electricity market
    {
        "name": ELEC_HIGH_VOLT_MARKET,
        "reference product": "electricity, high voltage",
        "unit": "kilowatt hour",
        "includes": [i for i in elec_scenario_variables.keys() if i not in ["Low to medium", "Medium to high"]],
        "replaces": [
            {
                "name": "market for electricity, high voltage",
                "product": "electricity, high voltage",
                "location": "CL"
            },
            {
                "name": "market for electricity, high voltage",
                "product": "electricity, high voltage",
                "location": "AR"
            }],
        "except regions": ["World"]
    },
    
    # Medium voltage electricity market
    {
        "name": ELEC_MEDIUM_VOLT_MARKET,
        "reference product": "electricity, medium voltage",
        "unit": "kilowatt hour",
        "includes": ["Medium to high"],
        "replaces": [
            {
                "name": "market for electricity, medium voltage",
                "product": "electricity, medium voltage",
                "location": "CL"
            },
            {
                "name": "market for electricity, medium voltage",
                "product": "electricity, medium voltage",
                "location": "AR"
            }],
        "except regions": ["World"]
    },

    # Low voltage electricity market
    {
        "name": ELEC_LOW_VOLT_MARKET,
        "reference product": "electricity, low voltage",
        "unit": "kilowatt hour",
        "includes": ["Low to medium"],
        "replaces": [
            {
                "name": "market for electricity, low voltage",
                "product": "electricity, low voltage",
                "location": "CL"
            },
            {
                "name": "market for electricity, low voltage",
                "product": "electricity, low voltage",
                "location": "AR"
            }],
        "except regions": ["World"],
    }
]

# Export config file
with open(CONFIG_FILE_PATH / "config_electricity.yaml", 'w') as yaml_file:
    yaml.dump(config_file_elec, yaml_file, default_flow_style=False)

### 1.2 Generate prospective LCI databases with premise

In [19]:
# Note: with premive v2.5.5, external scenarios are not incorporated in all databases
# database creation needs to be run one-by-one

dp_electricity = Package("../datapackage_electricity.json") # read the data package

scenarios = [
    # NDC scenarios
   # {"model": "remind", "pathway": "SSP2-NDC", "year": 2020, "external scenarios": [{"scenario": "STEPS", "data": dp_electricity}]},
    {"model": "remind", "pathway": "SSP2-NDC", "year": 2025, "external scenarios": [{"scenario": "STEPS", "data": dp_electricity}]},
   # {"model": "remind", "pathway": "SSP2-NDC", "year": 2030, "external scenarios": [{"scenario": "STEPS", "data": dp_electricity}]},
   # {"model": "remind", "pathway": "SSP2-NDC", "year": 2035, "external scenarios": [{"scenario": "STEPS", "data": dp_electricity}]},

    # SSP2-PkBudget1150 scenarios
  #  {"model": "remind", "pathway": "SSP2-PkBudg1150", "year": 2025, "external scenarios": [{"scenario": "APS", "data": dp_electricity}]},
  #  {"model": "remind", "pathway": "SSP2-PkBudg1150", "year": 2030, "external scenarios": [{"scenario": "APS", "data": dp_electricity}]},
   # {"model": "remind", "pathway": "SSP2-PkBudg1150", "year": 2035, "external scenarios": [{"scenario": "APS", "data": dp_electricity}]}
    ]

ndb = NewDatabase(
        scenarios=scenarios,
        source_db="ecoinvent-3.10-cutoff",
        source_version="3.10",
        key=PREMISE_KEY,
        use_absolute_efficiency=True,
        biosphere_name="ecoinvent-3.10-biosphere"
    )

ndb.update()

premise v.(2, 2, 5)
+------------------------------------------------------------------+
| Warning                                                          |
+------------------------------------------------------------------+
| Because some of the scenarios can yield LCI databases            |
| containing net negative emission technologies (NET),             |
| it is advised to account for biogenic CO2 flows when calculating |
| Global Warming potential indicators.                             |
| `premise_gwp` provides characterization factors for such flows.  |
| It also provides factors for hydrogen emissions to air.          |
|                                                                  |
| Within your bw2 project:                                         |
| from premise_gwp import add_premise_gwp                          |
| add_premise_gwp()                                                |
+------------------------------------------------------------------+
+-------------

Processing scenarios for all sectors: 100%|█| 1/1 [04:38<00:00, 278.02

Done!



In [20]:
ndb.write_db_to_brightway()

Write new database(s) to Brightway.
Running all checks...
Minor anomalies found: check the change report.


Writing activities to SQLite3 database:
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:56


Title: Writing activities to SQLite3 database:
  Started: 01/27/2025 19:52:54
  Finished: 01/27/2025 19:53:50
  Total time elapsed: 00:00:56
  CPU %: 96.50
  Memory %: 41.36
Created database: ei_cutoff_3.10_remind_SSP2-NDC_2025_STEPS 2025-01-27
Generate scenario report.
Report saved under c:\Users\istrateir\OneDrive - Universiteit Leiden\Research\lithium-prospective-scenarios\notebooks\export\scenario_report.
Generate change report.
Report saved under c:\Users\istrateir\OneDrive - Universiteit Leiden\Research\lithium-prospective-scenarios\notebooks.


## 3. Create foreground LCI databases

Here we create the lithium production LCI databases and link them to the corresponding background databases

In [4]:
# Define lithium market names
LI2CO3_MARKET_ALL = "market for lithium carbonate, from brine, S&P forecasts"
LI2CO3_MARKET_OTHER = "market for lithium carbonate, from brine, S&P forecasts, other projects"
LI2CO3_MARKET_PRODUCT = "lithium carbonate"

# Import lithium scenario data and mapping file between lithium variables and LCI datasets
scenario_data_lithium = pd.read_csv(LITHIUM_SCENARIO_DATA_PATH)
map_lithium_projects_to_lci = pd.read_excel(MAP_LITHIUM_LCIS_PATH)
map_lithium_projects_to_lci = dict(chain(zip(map_lithium_projects_to_lci['Project name'], map_lithium_projects_to_lci['LCI dataset'])))

# Map scenario variables to variable ID
lithium_scenario_variables = mapping_scenario_variables_to_ids(scenario_data_lithium)

In [ ]:
# Create the config file for lithium production
config_file_lithium = {}

# Populate production pathways
config_file_lithium["production pathways"] = {}

for var_id in lithium_scenario_variables:

    if var_id == "Other projects":
        # The "Other projects" category is modelled with the regional average
        # impact, therefore using the LI2CO3_MARKET_OTHER dataset
        var_lci = LI2CO3_MARKET_OTHER
        var_product = LI2CO3_MARKET_PRODUCT
        in_ecoinvent = False
        regionalize = True
        new_dataset = True

    else:
        var_lci = map_lithium_projects_to_lci[var_id]
        var_product = "df_rotary_dryer"
        regionalize = False
        new_dataset = False
        in_ecoinvent = False   
        
    populate_prod_pathaways(
        config_file=config_file_lithium, 
        var_id=var_id, 
        var_lci=var_lci, 
        var_product=var_product, 
        in_ecoinvent=in_ecoinvent, 
        regionalize=regionalize,
        new_dataset=new_dataset, 
        var=lithium_scenario_variables[var_id]
    )

# Populate markets
config_file_lithium["markets"] = [

    # Lithium carbonate market for other projects
    {
        "name": LI2CO3_MARKET_OTHER,
        "reference product": LI2CO3_MARKET_PRODUCT,
        "unit": "kilogram",
        "includes": [i for i in lithium_scenario_variables.keys() if i != "Other projects"],
        "except regions": ["World"]
    },

    # Lithium carbonate market
    {
        "name": LI2CO3_MARKET_ALL,
        "reference product": LI2CO3_MARKET_PRODUCT,
        "unit": "kilogram",
        "includes": [i for i in lithium_scenario_variables.keys()],
        "replaces": [
            {
                "name": "lithium carbonate production, from concentrated brine",
                "product": "lithium carbonate",
                "location": "GLO"
            }]
    }
]

# Export config file
with open(CONFIG_FILE_PATH / "config_lithium.yaml", 'w') as yaml_file:
    yaml.dump(config_file_lithium, yaml_file, default_flow_style=False)

In [5]:
# Create lithium production LCIs with premise

dp_lithium = Package("../datapackage_lithium.json") # read the data package
scenarios = [
    # S&P-Baseline 
    {"model": "remind", "pathway": "SSP2-NDC", "year": 2020, "external scenarios": [{"scenario": "S&P-Baseline", "data": dp_lithium}]},
    {"model": "remind", "pathway": "SSP2-NDC", "year": 2025, "external scenarios": [{"scenario": "S&P-Baseline", "data": dp_lithium}]},
    {"model": "remind", "pathway": "SSP2-NDC", "year": 2030, "external scenarios": [{"scenario": "S&P-Baseline", "data": dp_lithium}]},
    {"model": "remind", "pathway": "SSP2-NDC", "year": 2035, "external scenarios": [{"scenario": "S&P-Baseline", "data": dp_lithium}]},

    # S&P-Ambitious 
    {"model": "remind", "pathway": "SSP2-NDC", "year": 2025, "external scenarios": [{"scenario": "S&P-Ambitious", "data": dp_lithium}]},
    {"model": "remind", "pathway": "SSP2-NDC", "year": 2030, "external scenarios": [{"scenario": "S&P-Ambitious", "data": dp_lithium}]},
    {"model": "remind", "pathway": "SSP2-NDC", "year": 2035, "external scenarios": [{"scenario": "S&P-Ambitious", "data": dp_lithium}]},

    # S&P-Very Ambitious
    {"model": "remind", "pathway": "SSP2-NDC", "year": 2025, "external scenarios": [{"scenario": "S&P-Very Ambitious", "data": dp_lithium}]},
    {"model": "remind", "pathway": "SSP2-NDC", "year": 2030, "external scenarios": [{"scenario": "S&P-Very Ambitious", "data": dp_lithium}]},
    {"model": "remind", "pathway": "SSP2-NDC", "year": 2035, "external scenarios": [{"scenario": "S&P-Very Ambitious", "data": dp_lithium}]}
    ]

ndb = NewDatabase(
        scenarios=scenarios,
        source_db="ecoinvent-3.10-cutoff",
        source_version="3.10",
        key=PREMISE_KEY,
        use_absolute_efficiency=True,
        biosphere_name="ecoinvent-3.10-biosphere"
    )

# Update only external scenarios
ndb.update("external")

premise v.(2, 2, 5)
+------------------------------------------------------------------+
| Warning                                                          |
+------------------------------------------------------------------+
| Because some of the scenarios can yield LCI databases            |
| containing net negative emission technologies (NET),             |
| it is advised to account for biogenic CO2 flows when calculating |
| Global Warming potential indicators.                             |
| `premise_gwp` provides characterization factors for such flows.  |
| It also provides factors for hydrogen emissions to air.          |
|                                                                  |
| Within your bw2 project:                                         |
| from premise_gwp import add_premise_gwp                          |
| add_premise_gwp()                                                |
+------------------------------------------------------------------+
+-------------

Processing scenarios for sector 'external':   0%| | 0/10 [00:00<?, ?it

Extracted 1 worksheets in 0.26 seconds


Processing scenarios for sector 'external':  10%| | 1/10 [00:20<03:00,

Extracted 1 worksheets in 0.24 seconds


...

Extracted 1 worksheets in 0.23 seconds


Processing scenarios for sector 'external':  90%|▉| 9/10 [03:04<00:20,

Extracted 1 worksheets in 0.25 seconds


Processing scenarios for sector 'external': 100%|█| 10/10 [03:25<00:00

Done!



In [6]:
# Import lithium projects LCI datasets into wurst format
try:
  len(li_projects_lcis)
except NameError:
  li_projects_lcis = wurst.extract_brightway2_databases(LI_PROJECTS_DB)

Getting activity data


100%|██████████| 505/505 [00:00<00:00, 168332.16it/s]


Adding exchange data to activities


100%|██████████| 2611/2611 [00:00<00:00, 17344.08it/s]


Filling out exchange data


100%|██████████| 505/505 [00:00<00:00, 12461.16it/s]


In [7]:
# Extract lithium scenario databases
lithium_scenario_dbs = {}
for s, scenario in enumerate(ndb.scenarios):
    
    sc_name = ndb.scenarios[s]["external scenarios"][0]["scenario"]
    db_id = (sc_name, ndb.scenarios[s]["year"])
    
    scenario_db = load_database(scenario)
    market_ds = [ds for ds in ndb.scenarios[s]['database'] if ds["name"] in [LI2CO3_MARKET_ALL, LI2CO3_MARKET_OTHER]]
    lithium_scenario_dbs[db_id] = market_ds + li_projects_lcis

    for ds in lithium_scenario_dbs[db_id]:
        if "categories" in ds:
            ds.pop("categories")

In [8]:
# List of background databases
back_db_list = [db for db in bw.databases if "ei_cutoff_3.10_remind_SSP2" in db]

# Get datasets info in background databases
back_ds_info = {}
for db in back_db_list:
    print("Importing datasets for database:", db)
    back_ds_info[db] = {
        (ds["name"], ds["reference product"], ds["location"]): ds.key for ds in bw.Database(db)
    }

Importing datasets for database: ei_cutoff_3.10_remind_SSP2-NDC_2020_STEPS 2025-01-27
Importing datasets for database: ei_cutoff_3.10_remind_SSP2-PkBudg1150_2030_APS 2025-01-27
Importing datasets for database: ei_cutoff_3.10_remind_SSP2-PkBudg1150_2025_APS 2025-01-27
Importing datasets for database: ei_cutoff_3.10_remind_SSP2-PkBudg1150_2035_APS 2025-01-27
Importing datasets for database: ei_cutoff_3.10_remind_SSP2-NDC_2035_STEPS 2025-01-27
Importing datasets for database: ei_cutoff_3.10_remind_SSP2-NDC_2030_STEPS 2025-01-27
Importing datasets for database: ei_cutoff_3.10_remind_SSP2-NDC_2025_STEPS 2025-01-27


In [9]:
# Link lithium databases to background databases
technosphere = lambda x: x["type"] == "technosphere"
lithium_background_dbs = {}

for i in lithium_scenario_dbs:                   # Iterate over all litihum databases
    print("Update lithium scenario:", i)

    lithium_scenario = i[0]                      # Lithium scenario name
    year = str(i[1])                             # Year of assessment
    
    for back_db in back_db_list:                # Iterate over all background databases
        parts = back_db.split(' ')
        back_db_name = ' '.join(parts[0].split('_')[3:-1]) + ' ' + ' '.join(parts[1:-1])
        back_db_name = back_db_name.strip()
        
        # Proceed only if year matches for litihum and background databases
        if year in back_db_name:
            print("Background database:", back_db)

            li_db = copy.deepcopy(lithium_scenario_dbs[i])                          # Copy the lithium database
            lithium_db_name = "lithium_" + lithium_scenario + "_" + back_db_name    # Give a name to the lithium database according to the background

            # Update datasets input code to 
            for ds in li_db:
                for exc in filter(technosphere, ds["exchanges"]):
                    exc_data = (exc["name"], exc["product"], exc["location"])

                    if exc_data in back_ds_info[back_db]:
                        ds_code = back_ds_info[back_db][exc_data]
                        exc.update({'input': (ds_code[0], ds_code[1])})

            lithium_background_dbs[lithium_db_name] = li_db

    print("#########################")

Update lithium scenario: ('S&P-Baseline', 2020)
Background database: ei_cutoff_3.10_remind_SSP2-NDC_2020_STEPS 2025-01-27
#########################
Update lithium scenario: ('S&P-Baseline', 2025)
Background database: ei_cutoff_3.10_remind_SSP2-PkBudg1150_2025_APS 2025-01-27
Background database: ei_cutoff_3.10_remind_SSP2-NDC_2025_STEPS 2025-01-27
#########################
Update lithium scenario: ('S&P-Baseline', 2030)
Background database: ei_cutoff_3.10_remind_SSP2-PkBudg1150_2030_APS 2025-01-27
Background database: ei_cutoff_3.10_remind_SSP2-NDC_2030_STEPS 2025-01-27
#########################
Update lithium scenario: ('S&P-Baseline', 2035)
Background database: ei_cutoff_3.10_remind_SSP2-PkBudg1150_2035_APS 2025-01-27
Background database: ei_cutoff_3.10_remind_SSP2-NDC_2035_STEPS 2025-01-27
#########################
Update lithium scenario: ('S&P-Ambitious', 2025)
Background database: ei_cutoff_3.10_remind_SSP2-PkBudg1150_2025_APS 2025-01-27
Background database: ei_cutoff_3.10_remind_

In [10]:
# Link lithium datasets to regionalized water flows
for i in lithium_background_dbs:
    for ds in lithium_background_dbs[i]:
        project_name = ds["name"].rsplit('_', 1)[-1]
        relink_to_regionalized_water(water_flows_regionalized, ds, location=project_name)

In [11]:
for db in lithium_background_dbs:
    print("Writing database:", db)
    if db in bw.databases:
        del bw.databases[db]
    wurst.write_brightway2_database(lithium_background_dbs[db], db)
    print("==========================")

Writing database: lithium_S&P-Baseline_remind SSP2-NDC 2020
Vacuuming database 
514 datasets
2636 exchanges
0 unlinked exchanges
  


Writing activities to SQLite3 database:
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Title: Writing activities to SQLite3 database:
  Started: 04/05/2025 23:58:45
  Finished: 04/05/2025 23:58:45
  Total time elapsed: 00:00:00
  CPU %: 83.10
  Memory %: 34.17
Created database: lithium_S&P-Baseline_remind SSP2-NDC 2020
Writing database: lithium_S&P-Baseline_remind SSP2-PkBudg1150 2025
Vacuuming database 
514 datasets
2642 exchanges
0 unlinked exchanges
  


Writing activities to SQLite3 database:
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Title: Writing activities to SQLite3 database:
  Started: 04/06/2025 00:02:34
  Finished: 04/06/2025 00:02:34
  Total time elapsed: 00:00:00
  CPU %: 99.70
  Memory %: 26.81
Created database: lithium_S&P-Baseline_remind SSP2-PkBudg1150 2025
Writing database: lithium_S&P-Baseline_remind SSP2-NDC 2025
Vacuuming database 
514 datasets
2642 exchanges
0 unlinked exchanges
  


...

Title: Writing activities to SQLite3 database:
  Started: 04/06/2025 00:57:45
  Finished: 04/06/2025 00:57:45
  Total time elapsed: 00:00:00
  CPU %: 100.30
  Memory %: 5.81
Created database: lithium_S&P-Very Ambitious_remind SSP2-NDC 2030
Writing database: lithium_S&P-Very Ambitious_remind SSP2-PkBudg1150 2035
Vacuuming database 
518 datasets
2681 exchanges
0 unlinked exchanges
  


Writing activities to SQLite3 database:
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Title: Writing activities to SQLite3 database:
  Started: 04/06/2025 01:01:00
  Finished: 04/06/2025 01:01:00
  Total time elapsed: 00:00:00
  CPU %: 81.80
  Memory %: 5.82
Created database: lithium_S&P-Very Ambitious_remind SSP2-PkBudg1150 2035
Writing database: lithium_S&P-Very Ambitious_remind SSP2-NDC 2035
Vacuuming database 
518 datasets
2681 exchanges
0 unlinked exchanges
  


Writing activities to SQLite3 database:
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Title: Writing activities to SQLite3 database:
  Started: 04/06/2025 01:04:24
  Finished: 04/06/2025 01:04:24
  Total time elapsed: 00:00:00
  CPU %: 100.20
  Memory %: 5.83
Created database: lithium_S&P-Very Ambitious_remind SSP2-NDC 2035
